# Milestone 1

In [25]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt
import joblib

In [26]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout, Bidirectional, InputLayer
from tensorflow.keras.callbacks import EarlyStopping

In [27]:
sns.set(style="whitegrid", palette="coolwarm")
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)
random.seed(RANDOM_SEED)
print("Imports done")

Imports done


In [28]:
datasets = ["FD001","FD002","FD003","FD004"]

col_names = (['unit_number','time_in_cycles','op_setting_1','op_setting_2','op_setting_3']
             + [f'sensor_{i}' for i in range(1,22)])

In [29]:
def load_train_dataset(set_name):
    """
    Loads train_FDxxx.txt, computes RUL for each cycle, and attaches dataset_id
    Returns a dataframe
    """
    fname = f"train_{set_name}.txt"
    if not os.path.exists(fname):
        raise FileNotFoundError(f"{fname} not found in working dir.")
    df = pd.read_csv(fname, sep=' ', header=None)
    df.dropna(axis=1, how='all', inplace=True)
    df.columns = col_names
    max_cycle = df.groupby('unit_number')['time_in_cycles'].max().reset_index()
    max_cycle.columns = ['unit_number','max_cycle']
    df = df.merge(max_cycle, on='unit_number', how='left')
    df['RUL'] = df['max_cycle'] - df['time_in_cycles']
    df.drop('max_cycle', axis=1, inplace=True)
    df['dataset_id'] = set_name
    return df

In [30]:
print("Files needed:", [f"train_{d}.txt" for d in datasets])

Files needed: ['train_FD001.txt', 'train_FD002.txt', 'train_FD003.txt', 'train_FD004.txt']


In [31]:
dfs = []
for d in datasets:
    df_d = load_train_dataset(d)
    dfs.append(df_d)
combined = pd.concat(dfs, axis=0).reset_index(drop=True)
print("Combined shape:", combined.shape)
print(combined[['dataset_id','unit_number']].drop_duplicates().groupby('dataset_id').count())

combined.head()

Combined shape: (160359, 28)
            unit_number
dataset_id             
FD001               100
FD002               260
FD003               100
FD004               249


,unit_number,time_in_cycles,op_setting_1,op_setting_2,op_setting_3,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,...,sensor_14,sensor_15,sensor_16,sensor_17,sensor_18,sensor_19,sensor_20,sensor_21,RUL,dataset_id
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190,191,FD001
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236,190,FD001
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442,189,FD001
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739,188,FD001
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044,187,FD001


In [32]:
sensor_cols = [f'sensor_{i}' for i in range(1,22)]

scalers = {}
scaled_parts = []
for d in datasets:
    part = combined[combined['dataset_id']==d].copy()
    scaler = MinMaxScaler()
    part[sensor_cols] = scaler.fit_transform(part[sensor_cols])
    scalers[d] = scaler
    scaled_parts.append(part)
combined_scaled = pd.concat(scaled_parts, axis=0).reset_index(drop=True)

combined_scaled = pd.concat([combined_scaled, pd.get_dummies(combined_scaled['dataset_id'], prefix='ds')], axis=1)

for d, s in scalers.items():
    joblib.dump(s, f"scaler_{d}.pkl")

print("Per-dataset scaling complete. Example rows:")
combined_scaled.head()

Per-dataset scaling complete. Example rows:


,unit_number,time_in_cycles,op_setting_1,op_setting_2,op_setting_3,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,...,sensor_18,sensor_19,sensor_20,sensor_21,RUL,dataset_id,ds_FD001,ds_FD002,ds_FD003,ds_FD004
0,1,1,-0.0007,-0.0004,100.0,0.0,0.183735,0.406802,0.309757,0.0,...,0.0,0.0,0.713178,0.724662,191,FD001,True,False,False,False
1,1,2,0.0019,-0.0003,100.0,0.0,0.283133,0.453019,0.352633,0.0,...,0.0,0.0,0.666667,0.731014,190,FD001,True,False,False,False
2,1,3,-0.0043,0.0003,100.0,0.0,0.343373,0.369523,0.370527,0.0,...,0.0,0.0,0.627907,0.621375,189,FD001,True,False,False,False
3,1,4,0.0007,0.0000,100.0,0.0,0.343373,0.256159,0.331195,0.0,...,0.0,0.0,0.573643,0.662386,188,FD001,True,False,False,False
4,1,5,-0.0019,-0.0002,100.0,0.0,0.349398,0.257467,0.404625,0.0,...,0.0,0.0,0.589147,0.704502,187,FD001,True,False,False,False


In [33]:
def create_sequences(df, seq_len=30, sensor_cols=None, domain_onehot_prefix='ds_'):
    X, y, ids, ds = [], [], [], []
    grouped = df.groupby(['dataset_id','unit_number'])
    for (dataset_id, unit), group in grouped:
        group = group.sort_values('time_in_cycles')
        features = group[sensor_cols + [c for c in df.columns if c.startswith(domain_onehot_prefix)]].values
        labels = group['RUL'].values
        for i in range(len(features) - seq_len):
            X.append(features[i:i+seq_len])
            y.append(labels[i+seq_len])
            ids.append(unit)
            ds.append(dataset_id)
    return np.array(X), np.array(y), np.array(ids), np.array(ds)

In [34]:
seq_length = 30
domain_cols = [c for c in combined_scaled.columns if c.startswith('ds_')]
feature_cols = sensor_cols + domain_cols

X, y, x_unit_ids, x_ds = create_sequences(combined_scaled, seq_length, sensor_cols=feature_cols, domain_onehot_prefix='ds_')
print("Sequence shapes:", X.shape, y.shape)

Sequence shapes: (139089, 30, 29) (139089,)


In [35]:
print("Before fix:", X.dtype)
X = X.astype('float32')
y = y.astype('float32')
print("After fix:", X.dtype)

Before fix: object
After fix: float32
